In [2]:
import dask.distributed
import pystac_client
import planetary_computer
import stackstac
import shapely
import numpy as np
import pandas as pd
import rasterio
from matplotlib.colors import ListedColormap

In [3]:
metadata_df = pd.DataFrame({"chip_id", "lc", "latitude", "longitude"})

In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

http://127.0.0.1:8787/status


Task exception was never retrieved
future: <Task finished name='Task-41565' coro=<Client._gather.<locals>.wait() done, defined at /opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
2024-12-07 15:21:20,932 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:42679' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('asset-table-d0e54883500f07aa9fad40bc7d0dfba8', 0, 0)} (stimulus_id='handle-worker-cleanup-1733584880.9326236')
Task exception was never retrieved
future: <Task finished name='Task-47756' coro=<Client._gather.<locals>.wait() done, defined at /opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File

In [5]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [6]:
import leafmap
m = leafmap.Map(center=[13.85, 100.15], zoom=8, height="400px")
m

Map(center=[13.85, 100.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [7]:
polygon = m.user_rois['features'][0]['geometry']
bbox = shapely.geometry.Polygon(polygon["coordinates"][0]).bounds

In [8]:
chip_size = 20

In [9]:
search = catalog.search(collections = ["io-lulc-annual-v02"],
                        bbox = bbox,
                        datetime = "2023"
                       )

items = search.item_collection()
print(f"Returned {len(items)} Items")
lc_stack = stackstac.stack(
    items[0],
    dtype=np.ubyte,
    fill_value=255,
    sortby_date=False
).squeeze()
# m.add_geojson(items[0].to_dict(), zoom=12)

Returned 6 Items


In [10]:
def get_count(window, axis=None, **kwargs):
    window = window.compute()
    ret = np.zeros((window.shape[0], window.shape[2]))
    for i in range(0, window.shape[0]):
        for j in range(0, window.shape[2]):
            ret[i, j] = (np.unique(window[i, :, j, :])).shape[0]
    return ret

In [11]:
sub_lc_stack = lc_stack.isel(x=slice(9000, 10000), y=slice(9000, 10000))
lc_coarsen = sub_lc_stack.coarsen(x=chip_size, y=chip_size, boundary="trim")
lc_count = lc_coarsen.reduce(get_count)
indices = np.where(lc_count.values==1)

In [12]:
import tifffile as tiff

def save_array_as_tif(array, file_path):
    """
    Save a NumPy array as a TIFF file.

    Parameters:
        array (numpy.ndarray): The NumPy array to save.
        file_path (str): Path to save the TIFF file, including the .tif extension.

    Example:
        save_array_as_tif(my_array, "output_file.tif")
    """
    try:
        tiff.imwrite(file_path, array)
    except Exception as e:
        print(f"An error occurred while saving the array: {e}")

In [13]:
# Tif file creation
import scipy.misc
def gen_chips(array, index):
    output = np.zeros((224, 224))
    output[102:122,102:122] = array
    path = "/home/benchuser/data/chip_"
    save_array_as_tif(output, path + f"{index:04}"  + '.tif')


In [14]:
global_index = 0
for index, (i, j) in enumerate(zip(indices[0], indices[1])):
    array = lc_stack.isel(x=slice(i * chip_size, (i+1) * chip_size), y=slice(j * chip_size, (j+1) * chip_size))
    gen_chips(array, global_index)
    # metadata_df = metadata_df.concat([pd.DataFrame([[1,2]], columns=metadata_df.columns), metadata_df], ignore_index=True)
    global_index += 1

Array successfully saved as /home/benchuser/data/chip_0000.tif
Array successfully saved as /home/benchuser/data/chip_0001.tif
Array successfully saved as /home/benchuser/data/chip_0002.tif
Array successfully saved as /home/benchuser/data/chip_0003.tif
Array successfully saved as /home/benchuser/data/chip_0004.tif
Array successfully saved as /home/benchuser/data/chip_0005.tif
Array successfully saved as /home/benchuser/data/chip_0006.tif
Array successfully saved as /home/benchuser/data/chip_0007.tif
Array successfully saved as /home/benchuser/data/chip_0008.tif
Array successfully saved as /home/benchuser/data/chip_0009.tif
Array successfully saved as /home/benchuser/data/chip_0010.tif
Array successfully saved as /home/benchuser/data/chip_0011.tif
Array successfully saved as /home/benchuser/data/chip_0012.tif
Array successfully saved as /home/benchuser/data/chip_0013.tif
Array successfully saved as /home/benchuser/data/chip_0014.tif
Array successfully saved as /home/benchuser/data/chip_0

2024-12-07 15:21:20,921 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('asset_table_to_reader_and_window-fetch_raster_window-getitem-a65797b49f099018ba30c1ec784ce2d8', 0, 0))" coro=<Worker.execute() done, defined at /opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/worker_state_machine.py:3609>> ended with CancelledError


Array successfully saved as /home/benchuser/data/chip_0016.tif


KeyboardInterrupt: 

2024-12-07 15:21:22,929 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2024-12-07 15:21:22,931 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2024-12-07 15:21:22,931 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/opt/conda/envs/gfm_bench/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/gfm_bench/lib/python3.12/asyncio/base_events.py", line 686, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/nanny.py", line 984, in run
    await worker.finished()
  File "/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/core.py", line 491, in finished

In [ ]:
array.compute()